<a href="https://colab.research.google.com/github/MichaelMusembi/GRP17_Multimodal_Data-_Preprocessing/blob/main/multimodal-auth-system/notebook/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Data Preprocessing
This notebook merges customer social profiles with transaction data and creates features for ML.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
# Load datasets
social_profiles = pd.read_csv('../data/customer_social_profiles.csv')
transactions = pd.read_csv('../data/customer_transactions.csv')

# Clean column names
social_profiles.columns = social_profiles.columns.str.strip()
transactions.columns = transactions.columns.str.strip()

print(f"Social Profiles Shape: {social_profiles.shape}")
print(f"Transactions Shape: {transactions.shape}")

Social Profiles Shape: (155, 5)
Transactions Shape: (150, 6)


In [3]:
# Inspect data
print("Social Profiles:")
print(social_profiles.head())
print("\nTransactions:")
print(transactions.head())

Social Profiles:
   customer_id_new   social_media_platform  engagement_score  \
0  A178              LinkedIn                             74   
1  A190              Twitter                              82   
2  A150              Facebook                             96   
3  A162              Twitter                              89   
4  A197              Twitter                              92   

   purchase_interest_score review_sentiment  
0                      4.9         Positive  
1                      4.8          Neutral  
2                      1.6         Positive  
3                      2.6         Positive  
4                      2.3          Neutral  

Transactions:
   customer_id_legacy  transaction_id  purchase_amount   purchase_date  \
0                 151            1001              408   2024-01-01      
1                 192            1002              332   2024-01-02      
2                 114            1003              442   2024-01-03      
3          

In [4]:
# Extract numeric ID
social_profiles['customer_id_numeric'] = social_profiles['customer_id_new'].str.extract(r'(\d+)').astype(int)
print(social_profiles[['customer_id_new', 'customer_id_numeric']].head())

   customer_id_new  customer_id_numeric
0  A178                             178
1  A190                             190
2  A150                             150
3  A162                             162
4  A197                             197


In [5]:
# Merge datasets
merged_data = pd.merge(
    social_profiles,
    transactions,
    left_on='customer_id_numeric',
    right_on='customer_id_legacy',
    how='inner'
)

print(f"Merged Shape: {merged_data.shape}")
print(f"Columns: {list(merged_data.columns)}")

Merged Shape: (219, 12)
Columns: ['customer_id_new', 'social_media_platform', 'engagement_score', 'purchase_interest_score', 'review_sentiment', 'customer_id_numeric', 'customer_id_legacy', 'transaction_id', 'purchase_amount', 'purchase_date', 'product_category', 'customer_rating']


In [6]:
# Clean and convert data types
merged_data.columns = merged_data.columns.str.strip()

# Check data types and sample values
print("Data types:")
print(merged_data.dtypes)
print("\nSample customer_rating values:")
print(merged_data['customer_rating'].head(10))

Data types:
customer_id_new             object
social_media_platform       object
engagement_score             int64
purchase_interest_score    float64
review_sentiment            object
customer_id_numeric          int64
customer_id_legacy           int64
transaction_id               int64
purchase_amount              int64
purchase_date               object
product_category            object
customer_rating             object
dtype: object

Sample customer_rating values:
0                 3.8
1                 4.9
2                    
3                 3.6
4                 4.2
5                 1.4
6                 2.3
7                 3.2
8                 1.7
9                 4.8
Name: customer_rating, dtype: object


In [7]:
# Convert numeric columns
numeric_cols = ['customer_rating', 'purchase_amount', 'engagement_score', 'purchase_interest_score']
for col in numeric_cols:
    merged_data[col] = pd.to_numeric(merged_data[col], errors='coerce')

print("After conversion:")
print(merged_data[numeric_cols].dtypes)
print(f"\nMissing values: {merged_data[numeric_cols].isnull().sum()}")

After conversion:
customer_rating            float64
purchase_amount              int64
engagement_score             int64
purchase_interest_score    float64
dtype: object

Missing values: customer_rating            19
purchase_amount             0
engagement_score            0
purchase_interest_score     0
dtype: int64


In [8]:
# Aggregate by customer
customer_aggregated = merged_data.groupby('customer_id_new').agg({
    'social_media_platform': 'first',
    'engagement_score': 'first',
    'purchase_interest_score': 'first',
    'review_sentiment': 'first',
    'purchase_amount': ['sum', 'mean', 'count'],
    'customer_rating': 'mean',
    'product_category': lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0]
}).reset_index()

# Flatten column names
customer_aggregated.columns = ['_'.join(col).strip('_') if col[1] else col[0]
                                for col in customer_aggregated.columns.values]

print(f"Aggregated Shape: {customer_aggregated.shape}")
customer_aggregated.head()

Aggregated Shape: (61, 10)


,customer_id_new,social_media_platform_first,engagement_score_first,purchase_interest_score_first,review_sentiment_first,purchase_amount_sum,purchase_amount_mean,purchase_amount_count,customer_rating_mean,product_category_<lambda>
0,A100,Twitter,73,4.4,Negative,1118,279.500000,4,4.300000,Books
1,A101,Twitter,68,1.0,Neutral,1016,254.000000,4,2.525000,Books
2,A102,LinkedIn,51,4.8,Neutral,569,189.666667,3,3.433333,Electronics
3,A103,Instagram,77,2.3,Positive,2391,265.666667,9,4.200000,Sports
4,A104,Twitter,91,4.6,Negative,2094,349.000000,6,1.450000,Clothing


In [9]:
# Encode categorical variables
le_platform = LabelEncoder()
le_sentiment = LabelEncoder()
le_category = LabelEncoder()

customer_aggregated['platform_encoded'] = le_platform.fit_transform(customer_aggregated['social_media_platform_first'])
customer_aggregated['sentiment_encoded'] = le_sentiment.fit_transform(customer_aggregated['review_sentiment_first'])
customer_aggregated['category_encoded'] = le_category.fit_transform(customer_aggregated['product_category_<lambda>'])

customer_aggregated.head()

,customer_id_new,social_media_platform_first,engagement_score_first,purchase_interest_score_first,review_sentiment_first,purchase_amount_sum,purchase_amount_mean,purchase_amount_count,customer_rating_mean,product_category_<lambda>,platform_encoded,sentiment_encoded,category_encoded
0,A100,Twitter,73,4.4,Negative,1118,279.500000,4,4.300000,Books,4,0,0
1,A101,Twitter,68,1.0,Neutral,1016,254.000000,4,2.525000,Books,4,1,0
2,A102,LinkedIn,51,4.8,Neutral,569,189.666667,3,3.433333,Electronics,2,1,2
3,A103,Instagram,77,2.3,Positive,2391,265.666667,9,4.200000,Sports,1,2,4
4,A104,Twitter,91,4.6,Negative,2094,349.000000,6,1.450000,Clothing,4,0,1


In [11]:
# Save results
customer_aggregated.to_csv('../data/merged_customer_data.csv', index=False)
merged_data.to_csv('../data/merged_customer_data_detailed.csv', index=False)

print("Files saved successfully!")
print(f"Final dataset shape: {customer_aggregated.shape}")
print(f"Columns: {list(customer_aggregated.columns)}")

Files saved successfully!
Final dataset shape: (61, 13)
Columns: ['customer_id_new', 'social_media_platform_first', 'engagement_score_first', 'purchase_interest_score_first', 'review_sentiment_first', 'purchase_amount_sum', 'purchase_amount_mean', 'purchase_amount_count', 'customer_rating_mean', 'product_category_<lambda>', 'platform_encoded', 'sentiment_encoded', 'category_encoded']
